In [1]:
 pip install chromadb sentence-transformers google-genai neo4j -U bitsandbytes accelerate bert-score RapidFuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from neo4j import GraphDatabase
import json, torch, gc, chromadb, os, pickle
from collections import defaultdict
from bert_score import score as bert_score
from bert_score import BERTScorer
from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions
from tqdm import tqdm
from google import genai
from neo4j import GraphDatabase
from google.genai import types

2025-08-25 16:04:45.055501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756137885.414567      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756137885.514626      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch, re

# Load model + tokenizer
llm_path = 'AITeamVN/Vi-Qwen2-7B-RAG'
tokenizer = AutoTokenizer.from_pretrained(llm_path, trust_remote_code=True)

max_memory = {
    0: "14GiB",  # GPU 0
    1: "14GiB",  # GPU 1
    "cpu": "30GiB" 
}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # hoặc float16 nếu gặp lỗi
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    "AITeamVN/Vi-Qwen2-7B-RAG",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("AITeamVN/Vi-Qwen2-7B-RAG", use_fast=False)

# model = torch.compile(model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [4]:
# Precedent database
PRECEDENT_DB_URI = "neo4j+s://06cb763b.databases.neo4j.io"
PRECEDENT_DB_AUTH = ("neo4j", "h9ApWqGVBKuEv3JFuuw7D1VHDPm_DdwE-GQ__7nkwqY")

precedent_driver = GraphDatabase.driver(PRECEDENT_DB_URI, auth=PRECEDENT_DB_AUTH)
precedent_driver.verify_connectivity()
print("Connected to Precedent DB.")

# Law database
LAW_DB_URI = "neo4j+s://504fffc1.databases.neo4j.io"
LAW_DB_AUTH = ("neo4j", "yqAInILZisA5ZYCxJX6s8dSfmae7Th22KxLGysRN65M")

law_driver = GraphDatabase.driver(LAW_DB_URI, auth=LAW_DB_AUTH)
law_driver.verify_connectivity()
print("Connected to Law DB.")

Connected to Precedent DB.
Connected to Law DB.


# Án lệ

In [ ]:
import chromadb
precedentEmbedModel = SentenceTransformer("truro7/vn-law-embedding", device='cuda')
client = chromadb.Client()
collection_name = "vn_precedent_docs"

existing_collections = [col.name for col in client.list_collections()]
if collection_name in existing_collections:
    client.delete_collection(name=collection_name)

# 5. Tạo mới collection
collection = client.create_collection(name=collection_name)

file_path = "/kaggle/input/precedent-embedding/precedent_embeddings.jsonl"
texts, embeddings, metadata_list = [], [], []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        texts.append(data["text"])
        embeddings.append(data["embedding"])
        metadata_list.append(data["metadata"])

# Add vào Chroma theo batch
def batch_iter(data, meta, emb, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size], meta[i:i+batch_size], emb[i:i+batch_size], i

batch_size = 2000
for batch_texts, batch_meta, batch_emb, start_idx in tqdm(batch_iter(texts, metadata_list, embeddings, batch_size)):
    batch_ids = [f"id_{i}" for i in range(start_idx, start_idx + len(batch_texts))]
    collection.add(
        documents=batch_texts,
        embeddings=batch_emb,
        ids=batch_ids,
        metadatas=batch_meta
    )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.
1it [00:00,  7.98it/s]


In [ ]:
lawEmbedModel = SentenceTransformer("AITeamVN/Vietnamese_Embedding_v2",device='cuda')
client = chromadb.Client()
collection_name = "vn_law_docs"

existing_collections = [col.name for col in client.list_collections()]
if collection_name in existing_collections:
    client.delete_collection(name=collection_name)

# 5. Tạo mới collection
collection = client.create_collection(name=collection_name)

file_path = "/kaggle/input/law-embedding/law_embeddings.jsonl"
texts, embeddings, metadata_list = [], [], []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        texts.append(data["text"])
        embeddings.append(data["embedding"])
        metadata_list.append(data["metadata"])
        
# Thêm vào Chroma theo batch
def batch_iter(data, meta, emb, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size], meta[i:i+batch_size], emb[i:i+batch_size], i

batch_size = 2000
for batch_texts, batch_meta, batch_emb, start_idx in tqdm(batch_iter(texts, metadata_list, embeddings, batch_size)):
    batch_ids = [f"law_id_{i}" for i in range(start_idx, start_idx + len(batch_texts))]
    collection.add(
        documents=batch_texts,       
        embeddings=batch_emb,
        ids=batch_ids,
        metadatas=batch_meta
    )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

16it [00:46,  2.90s/it]


In [8]:
def suit_precedent(result):
    client = genai.Client(api_key=GEMINI_API_KEY)
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction="Hãy đánh giá và so sánh tình huống pháp lý trong yêu cầu của người dùng với các án lệ được truy xuất. Tập trung vào mức độ tương đồng về tình tiết, sự kiện pháp lý và vấn đề pháp lý được giải quyết trong án lệ. Chỉ trả về 1 mã án lệ phù hợp nhất (ví dụ: 01/2016/AL, 02/2016/AL, 63/2022/AL) nếu tình huống người dùng và án lệ có sự tương đồng rõ ràng về mặt pháp lý. Nếu không có án lệ nào đủ tương đồng, trả về một xâu rỗng. Không giải thích, không lập luận, không kèm thêm nội dung nào khác ngoài mã án lệ hoặc xâu rỗng."),
        contents=result
    )
    
    data = response.text.strip() if response.text else ""

    return data if data else None
    
def insert_summary(tx, case_number, summary_text):
    tx.run("""
        MERGE (c:Case {case_number: $case_number})
        CREATE (s:Summary {text: $summary})
        MERGE (c)-[:HAS_SUMMARY]->(s)
    """, case_number=case_number, summary=summary_text)

with precedent_driver.session() as session:
    with open("/kaggle/input/summary-an-le/summary_an_le.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            if data.get("section") == "Nội dung vụ án":
                session.execute_write(insert_summary, data["case_number"], data["content"])

In [9]:
import re

def build_context(precedent_data: list, applied_laws: dict) -> str:
    context_parts = []
    for precedent in precedent_data:
        case_number = precedent.get("case_number", "Không rõ mã án lệ")
        summary = precedent.get("summary", "")
        decision = precedent.get("decision", "")
        core = precedent.get("core", "")
        
        text = f"Án lệ: {case_number}\n" \
               f"- Tóm tắt: {summary}\n" \
               f"- Quyết định: {decision}\n" \
               f"- Nội dung án lệ: {core}\n"

        # Ghép điều luật tương ứng nếu có
        laws = applied_laws.get(case_number, [])
        if laws:
            text += "- Các luật được dùng:\n"
            for law in laws:
                law_text = f"  + Điều luật: {law['title']} ({law['law']})\n" \
                           f"    Nội dung: {law['content']}"
                text += law_text + "\n"

        context_parts.append(text.strip())

    return "\n\n".join(context_parts)

def get_precedent_answer(query: str, precedent_data: list, applied_laws: list, max_new_tokens: int = 2048) -> str:
    context = build_context(precedent_data, applied_laws)
    
    # Prompt hướng dẫn
    SYSTEM_PROMPT = '''Bạn là một trợ lý pháp lý. Khi người dùng cung cấp một tình huống pháp lý và đặt câu hỏi, hoặc kèm theo thông tin tham khảo như điều luật, án lệ, văn bản pháp luật, hãy trả lời theo các nguyên tắc sau:
    1. Ngôn ngữ: Đảm bảo trả lời hoàn toàn bằng *tiếng Việt, đúng chính tả, không sử dụng từ ngữ nước ngoài.*
    2.Nội dung:
    Trả lời ngắn gọn, chính xác, bám sát câu hỏi.
    Không tự suy diễn hoặc thêm tình tiết không có trong tình huống.
    Không sử dụng lời chào, văn phong cảm tính hay hành chính.
    3. Sử dụng căn cứ pháp lý:
    Nếu áp dụng điều luật, phải nêu rõ số điều và trích nguyên văn nội dung áp dụng.
    Nếu sử dụng án lệ, nêu rõ số án lệ, nội dung cốt lõi và lý do liên quan đến tình huống.
    4. Nếu thiếu thông tin hoặc không có căn cứ pháp lý rõ ràng:
    Trả lời: “Không đủ thông tin để kết luận” hoặc “Không có căn cứ pháp lý rõ ràng để áp dụng trong trường hợp này.”
    5. Không đưa lời khuyên cảm tính hoặc đạo đức.
    Không sử dụng các cụm như “nên làm”, “cần phải” nếu không có căn cứ pháp lý cụ thể.'''
    
    TEMPLATE = """
    Tình huống pháp lý:
    {question}
    
    Các căn cứ pháp lý có thể áp dụng (án lệ, điều luật...):
    {context}
    
    **Hãy cung cấp trả lời bám sát và phù hợp với câu hỏi của người dùng**
    """

    conversation = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": TEMPLATE.format(context=context, question=query)}
    ]

    text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
 
    generated_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
    )

    print("Token length:", len(inputs['input_ids'][0]))
    generated_ids = [output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, generated_ids)]
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    cleaned = re.split(r"(### Câu hỏi:)", decoded)[0]
    return cleaned.strip()

In [10]:
def get_precedent(precedent_list):
    result = []
    for precedent_number in precedent_list:
        records, summary, keys = precedent_driver.execute_query(
            """
            MATCH (c:Case {case_number: $precedent_number})
            OPTIONAL MATCH (c)-[:HAS_SUMMARY]->(sm:Summary)
            OPTIONAL MATCH (c)-[:HAS_DECISION]->(dc:Decision)
            OPTIONAL MATCH (c)-[:HAS_PRECEDENT_CONTENT]->(pc:PrecedentContent)
            RETURN 
              c.case_number AS case_number,
              head(collect(sm.text)) AS summary,
              head(collect(dc.text)) AS decision,
              head(collect(pc.text)) AS core
            """,
            precedent_number=precedent_number
        )
        for record in records:
            result.append({"case_number": record["case_number"],"summary": record.get("summary", ""),
                           "decision": record.get("decision", ""),"core": record.get("core", "")})
    return result

def get_applied_law(precedent_list):
    result = {}
    for precedent_number in precedent_list:
        records, _, _ = law_driver.execute_query(
            """
            MATCH (c:Case {case_number: $precedent_number})-[:HAS_APPLIED]->(n:Article)
            MATCH (n)-[:HAS_CHUNK]->(a:Chunk)
            WHERE n.code = a.code
            WITH n.title AS title, n.code AS law_name, collect(a.content) AS chunks
            RETURN law_name, title, apoc.text.join(chunks, " ") AS full_content
            UNION
            MATCH (c:Case {case_number: $precedent_number})-[:HAS_APPLIED]->(n:Statute)
            MATCH (n)-[:HAS_CHUNK]->(a:Chunk)
            WHERE n.code = a.code
            WITH n.title AS title, n.code AS law_name, collect(a.content) AS chunks
            RETURN law_name, title, apoc.text.join(chunks, " ") AS full_content
            """,
            precedent_number=precedent_number
        )
        laws = []
        for record in records:
            laws.append({
                "law": record["law_name"],
                "title": record["title"],
                "content": record["full_content"]
            })
        result[precedent_number] = laws
    return result

In [11]:
def retrieve_precedent(query):
    client = chromadb.Client()
    collection=client.get_collection("vn_precedent_docs")
    query_embedding = precedentEmbedModel.encode([query])[0]
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )
    
    THRESHOLD = 1

    top_results = results["documents"][0]
    distances = results["distances"][0]
    metadatas = results["metadatas"][0]

    print(sum(distances)/len(distances))
    
    filtered = [
        {
            "case_number": metadatas[i]["case_number"],
            "document": top_results[i],
            "distance": distances[i]
        }
        for i in range(len(top_results))
        if distances[i] <= THRESHOLD
    ]
    
    if not filtered:
        return "None"
    
    results_text = "\n\n".join(
        f"Án lệ {r['case_number']} (distance: {r['distance']:.4f}):\n{r['document']}"
        for r in filtered
    )
    
    final_prompt = f"Tình huống:\n{query}\n\nCác án lệ phù hợp:\n{results_text}"
    return final_prompt

# LAW

In [12]:
from google.genai import types
def suit_question(query):
    client = genai.Client(api_key=GEMINI_API_KEY)
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction="""
            Bạn là chuyên gia pháp lý Việt Nam. 
            Nhiệm vụ: Phân tích tình huống pháp lý được cung cấp và đưa ra 3–5 câu hỏi ngắn gọn, cần thiết để tra cứu các luật dùng để giải quyết cho tình huống.
            Chỉ hỏi về quy định pháp luật chung, không đề cập tên riêng, nghề nghiệp hoặc số liệu cụ thể trong tình huống.
            Yêu cầu:
            - Chỉ viết câu hỏi, không giải thích.
            - Ngắn gọn, tối đa 20 từ mỗi câu.
            - Không hỏi về chứng cứ hay thủ tục hành chính, chỉ tập trung vào vấn đề pháp lý cốt lõi.
            - Chỉ dùng tiếng Việt đúng chính tả.
            Tình huống:
            """),
        contents=query
    )

    data=response.text
    data=[x.strip() for x in data.split('\n') if x.strip()]
    return data

def query_law(query):
    client = chromadb.Client()
    all_law=[]
    collection=client.get_collection("vn_law_docs")

    for q in query:
        query_embedding = lawEmbedModel.encode([q])[0]
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=5
        )
        all_law.append(results)
    return all_law

In [13]:
def normalized_law(all_results):
    normalized_entries = []
    seen = set()

    for results in all_results:
        distances = results["distances"][0]
        metadatas = results["metadatas"][0]

        for law, statute, distance in zip(
            [m["law"] for m in metadatas],
            [m["statute"] for m in metadatas],
            distances
        ):
            key_tuple = (law, statute)
            if key_tuple in seen:
                continue
            seen.add(key_tuple)

            if "Bộ luật" in law:
                normalized_key = f"{law.replace(' ', '_')}_{statute.replace(' ', '').split('.')[0]}"
                is_code = True
            else:
                normalized_key = f"{law} {statute}"
                is_code = False

            normalized_entries.append({
                "key": normalized_key,
                "is_code": is_code,
                "law": law.strip(),
                "statute": statute.strip(),
                "distance": distance
            })
    return normalized_entries

In [14]:
from rapidfuzz import fuzz
def dedup_by_embedding(results, threshold=0.9):
    selected = []
    for r in results:
        is_dup = False
        for s in selected:
            sim = fuzz.token_set_ratio(r["content"], s["content"])
            if sim >= threshold:
                # Nếu luật mới "tốt hơn" thì thay thế
                if r["distance"] < s["distance"]:
                    s.update(r)
                is_dup = True
                break
        if not is_dup:
            selected.append(r)
    return selected

def retrieve_laws(normalized_laws):
    results = []

    for i, item in enumerate(normalized_laws):
        key = item["key"]
        is_code = item["is_code"]
        law = item["law"]
        statute = item["statute"]
        distance = item["distance"]

        id_pattern = f"(?i).*{key}.*"

        records, _, _ = driver.execute_query(
            """
            MATCH (a:Article)-[:HAS_CHUNK]->(ch:Chunk)
            WHERE ch.id =~ $id_pattern AND a.title=$statute
            WITH a, ch
            ORDER BY ch.index
            RETURN a.code AS LawTitle,a.title AS ArticleTitle, collect(ch.content) AS Chunks
            UNION
            MATCH (s:Statute)-[:HAS_CHUNK]->(ch:Chunk)
            WHERE s.law = $law AND s.title CONTAINS $statute
            RETURN s.law AS LawTitle,s.title AS ArticleTitle, collect(ch.content) AS Chunks
            """,
            id_pattern=id_pattern,law=law,statute=statute
        )

        for rec in records:
            content = " ".join(rec["Chunks"]).strip()
            results.append({
                "LawTitle": rec["LawTitle"],
                "ArticleTitle": rec["ArticleTitle"],
                "content": content,
                "distance": distance
            })
    
    deduped = dedup_by_embedding(results, threshold=0.90)
    filtered= [r for r in results if r["distance"] <= 0.95]
    context_str = "\n".join(
        f"{r['ArticleTitle']}, {r['LawTitle']}: {r['content']}"
        for r in filtered
    )
    return context_str, filtered

In [15]:
def get_law_answer(law_list, user_query: str, max_new_tokens=1024):
    # Prompt hướng dẫn
    SYSTEM_PROMPT = '''Bạn là một trợ lý pháp lý. Khi người dùng cung cấp một tình huống pháp lý và 
    đặt câu hỏi, hoặc kèm theo thông tin tham khảo như điều luật, án lệ, văn bản pháp luật, 
    hãy trả lời theo các nguyên tắc sau:
    1. Ngôn ngữ: Đảm bảo trả lời hoàn toàn bằng *tiếng Việt, đúng chính tả, không sử dụng từ ngữ 
    nước ngoài.*
    2.Nội dung:
    Trả lời ngắn gọn, chính xác, bám sát câu hỏi.
    Không tự suy diễn hoặc thêm tình tiết không có trong tình huống.
    Không sử dụng lời chào, văn phong cảm tính hay hành chính.
    3. Sử dụng căn cứ pháp lý:
    Nếu áp dụng điều luật, phải nêu rõ số điều và trích nguyên văn nội dung áp dụng.
    Nếu sử dụng án lệ, nêu rõ số án lệ, nội dung cốt lõi và lý do liên quan đến tình huống.
    Dùng điều luật mới nhất (ví dụ: dùng bộ luật dân sự 2015 thay vì dùng bộ luật dân sự 2005)
    **Trong các điều luật được đưa, phải xem xét các điều luật nào là phù hợp để giải quyết 
    tình huống**
    4. Nếu thiếu thông tin hoặc không có căn cứ pháp lý rõ ràng:
    Trả lời: “Không đủ thông tin để kết luận” hoặc “Không có căn cứ pháp lý rõ ràng để áp dụng trong 
    trường hợp này.”
    5. Không đưa lời khuyên cảm tính hoặc đạo đức.
    Không sử dụng các cụm như “nên làm”, “cần phải” nếu không có căn cứ pháp lý cụ thể.
    '''
    
    TEMPLATE = """
    Tình huống pháp lý:
    {question}
    
    Các căn cứ pháp lý có thể áp dụng (án lệ, điều luật...):
    {context}
    
    **Hãy cung cấp trả lời bám sát và phù hợp với câu hỏi của người dùng**
    """
    conversation = [
        {"role": "system", "content": SYSTEM_PROMPT.strip()},
        {"role": "user", "content": TEMPLATE.format(
            question=user_query.strip(),
            context=law_list
        )}
    ]
    
    text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    
    generated_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    generated_ids = [output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, generated_ids)]
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    cleaned = re.split(r"(### Câu hỏi:)", decoded)[0].strip()
    return cleaned

# BACKEND

In [ ]:
# 1. Cài thư viện
!pip install -q fastapi uvicorn nest_asyncio pyngrok streamlit
from pyngrok import ngrok
import threading
import nest_asyncio
nest_asyncio.apply()

# 2. Set Ngrok token
ngrok.set_auth_token("2efGJiQaVuc2T3ZvY7GML1DAcl1_51kuQumr7HJrwwUhcujGf")

# 3. Import các thành phần backend
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# 4. Tạo app FastAPI
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 5. Định nghĩa schema
class Message(BaseModel):
    message: str

# 6. Định nghĩa route
@app.get("/")
def read_root():
    return {"message": "Backend đang chạy!"}

@app.post("/chat")
def chat_handler(query: Message):
    try:
        user_query = query.message  # lấy string từ Pydantic object
        print("-" * 100)
        print("Câu hỏi từ UI:", user_query)
        
        top5 = retrieve_precedent(user_query)
        precedent_code = suit_precedent(top5)
        print(precedent_code)
        
        if precedent_code:
            print(5*"-", "Dùng án lệ", 5*"-")
            precedents = get_precedent([precedent_code])
            laws = get_applied_law([precedent_code])
            answer = get_precedent_answer(
                query=user_query,   
                precedent_data=precedents,
                applied_laws=laws
            )
            print(answer)
            return {"answer": answer}
        else:
            print(5*"-", "Dùng luật", 5*"-")
            questions = suit_question(user_query)  
            laws = query_law(questions)
            normalized = normalized_law(laws)
            laws_content = retrieve_laws(normalized)
            response = get_law_answer(laws_content, user_query)
            print(response)
            return {"answer": response}

    except Exception as e:
        print("Lỗi backend:", e)
        return {"answer": f"Lỗi hệ thống: {str(e)}"}


# 8. Khởi tạo Ngrok
public_url = ngrok.connect(8000)
print(f"🎯 Public FastAPI endpoint: {public_url}/chat")

# Anh Nam muốn mua một căn nhà của ông Hùng với giá 3 tỷ đồng. Hai bên đã thỏa thuận miệng và anh Nam đã đặt cọc cho ông Hùng số tiền 300 triệu đồng, có lập văn bản đặt cọc rõ ràng. Trong văn bản có ghi, sau 30 ngày kể từ ngày đặt cọc, hai bên sẽ tiến hành ký hợp đồng mua bán công chứng. Tuy nhiên, sau 20 ngày, giá đất khu vực đó tăng mạnh. Ông Hùng thông báo không bán nhà cho anh Nam nữa và muốn trả lại 300 triệu tiền cọc. Anh Nam không đồng ý và yêu cầu ông Hùng phải bồi thường. 
# Hỏi yêu cầu của anh Nam có đúng không và ông Hùng phải chịu trách nhiệm như thế nào?
uvicorn.run(app, host="0.0.0.0", port=8000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.8 MB/s eta 0:00:00:00:01
🎯 Public FastAPI endpoint: NgrokTunnel: "https://f1cee5182c50.ngrok-free.app" -> "http://localhost:8000"/chat


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
